In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
import time
import re

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

service = Service()
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(10)
driver.maximize_window()

wait = WebDriverWait(driver, 10)

action = ActionChains(driver)

driver.get(
    "https://www.youtube.com/watch?v=zA-F9EtsRJc&list=PLuMuHAJh9g_Py_PSm8gmHdlcil6CQ9QCM"
)

titles = []
playlists = driver.find_element(
    By.XPATH,
    "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[2]/div/ytd-playlist-panel-renderer/div/div[3]",
)
title_elems = playlists.find_elements(By.ID, "video-title")
# title_elems = title_elems.find_elements(By.XPATH,'')
link_imgs = playlists.find_elements(By.CSS_SELECTOR, "img")
for title in title_elems:
    title = title.text.split("l ")[-1].split(" (with.")
    title, guest = title[0], title[1].split(")")[0] if len(title) != 1 else ""
    title = title.split(" ", 1)
    loc, food = title[0], title[1] if len(title) != 1 else ""
    titles.append([loc, food, guest])

for i, link in enumerate(link_imgs):
    action.move_to_element(link).perform()
    link.click()

    time.sleep(1)
    try:
        premium = driver.find_element(By.XPATH,'//*[@id="dismiss-button"]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]')
        premium.click()
    except Exception as e:
        pass
    expand_button = driver.find_element(By.ID, "expand")
    expand_button.click()
    # 조회수
    views = driver.find_element(By.XPATH, '//*[@id="info"]/span[1]')
    views = int(re.sub(r"[가-힣,\s]", "", views.text))
    titles[i].append(views)
    # DATETIME
    date = driver.find_element(By.XPATH, '//*[@id="info"]/span[3]')
    date = re.sub(r"[가-힣,.:]", "", date.text).strip().split()
    for d in date:
        titles[i].append(d)

    button = driver.find_element(
        By.XPATH, '//*[@id="primary-button"]/ytd-button-renderer/yt-button-shape/button'
    )
    action.move_to_element(button).perform()
    button.click()

    wait.until(EC.presence_of_element_located((By.ID, "secondary")))
    time.sleep(5)

    # 페이지 소스 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 자막 세그먼트 모두 가져오기
    segments = soup.find_all(
        "div", class_="segment style-scope ytd-transcript-segment-renderer"
    )

    # 자막 시간과 텍스트 추출
    texts = list()

    for segment in segments:
        text = segment.find(
            "yt-formatted-string",
            class_="segment-text style-scope ytd-transcript-segment-renderer",
        ).get_text(strip=True)
        texts.append(text)

    titles[i].append(texts)


driver.quit()

['2024', '9', '10']


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.126)
Stacktrace:
#0 0x562604ba4c5a <unknown>
#1 0x562604887e2c <unknown>
#2 0x56260485d0d4 <unknown>
#3 0x5626048ffa8d <unknown>
#4 0x562604915ca6 <unknown>
#5 0x5626048f7363 <unknown>
#6 0x5626048c7247 <unknown>
#7 0x5626048c7b9e <unknown>
#8 0x562604b6b22b <unknown>
#9 0x562604b6f2d1 <unknown>
#10 0x562604b56ade <unknown>
#11 0x562604b6fe32 <unknown>
#12 0x562604b3b77f <unknown>
#13 0x562604b94618 <unknown>
#14 0x562604b947f0 <unknown>
#15 0x562604ba3d8c <unknown>
#16 0x7f37b6d3f609 start_thread


In [5]:
print(titles[0])

['인현시장', '실비식당', '노래 콜라보 이벤트 당첨자', 514304, ':2024.9.10.', ["[SUNG SI KYUNG's Grab a Bite]", "It's a mecca for good restaurants. Gangbuk.", 'Inhyeon Market.', "I'm too big.", "And people recognize me since I'm on TV and stuff.", "So I don't get to visit places like here.", 'Every time I come here, I feel so excited', 'and I want to visit all the places.', 'Is this the silver pomfret street?', 'There are a lot of silver pomfret.', 'Andongjip.', "I featured 'Andongjip' in Gyeongdong Market once.", 'I heard that they sell Baechu Jeon.', 'So many.', 'I want to visit them all. I came here today because...', 'Among the juniors', 'who participated in the song', 'Nayeon said', "that she really wanted to visit this place called 'Silbi Sikdang.' Ah!", 'She said that she visited this place a couple of times and that it was great.', 'I saw the place', "on Instagram a couple of times so I've been curious about it too.", 'It looked quite unique.', 'It was very Instagrammable.', 'The prices and', 'the

In [2]:
import pandas as pd

df = pd.DataFrame(
    titles,
    columns=["title", "food", "guest", "views", "year", "month", "day", "script"],
)
print(df)

ValueError: 8 columns passed, passed data had 10 columns

In [ ]:
df.to_csv("sungsi.csv")